This is an example run through to show how the match_audio_to_video workflow goes


Author : Thejasvi Beleyur, August 2019,
         Acoustic and Functional Ecology Group, Max Planck Institute for Ornithology, Seewiesen

In [1]:
import sys 
sys.path.append('../bin/') # include the modules in the outer folder in the search path
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 
import datetime as dt
import os


In [2]:
%matplotlib qt

In [3]:
import Tkinter

In [4]:
Tkinter.__version__

'$Revision: 81008 $'

### Getting your borders right:
For reliable and quick light intensity tracking and timestamp reading it is best to provide a narrow region around the focal area. In my experience I've seen that the timestamp reading can get a bit wonky if the whole image is provided!

In [6]:
# Get the led and timestamp borders 
%run ../bin/browse_through_video.py -v "./vlc-record-2019-08-19-16h32m09s-DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi"

change frame


So - for the example video the led_border entry should be (418.09, 792.16, 492.46, 246.62) *your points may be different than mine!!* - don't bother with too many decimal places! - repeat this once more to get the timestamp_border - and do this for every video file you have! 
The points are chosen by left click. If you want to delete an accidental click then use the right click. 
If you think the 4 points you chose still aren't right - no worries you can press the choose border button once more and redo the whole process. 

You could redo the border selection any time by keeping the frame-by-frame display open!

The border output you need to put into the annotations file is always printed out. 


## Getting light intensity and timestamps in the video:
After entering your annotations, now run the  routine that will measure the light intensity in each frame and read the timestamp. 

In [8]:
from generate_data_from_video import generate_videodata_from_videofiles

annotations_df = pd.read_csv('eg_annotations.csv')
# Since it's a small video with only 1370 frames - we'll run the whole thing! This could take a couple of minutes
generate_videodata_from_videofiles(annotations_df)

Frames read:   0%|          | 0/895 [00:00<?, ?it/s]

('Now generating dat from row', 0)
gettin raw video data from vlc-record-2019-08-19-16h32m09s-DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi  now....
./vlc-record-2019-08-19-16h32m09s-DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi
starting frame reading


Frames read: 100%|██████████| 895/895 [03:19<00:00,  4.47it/s]
1it [03:19, 199.95s/it]

Done with frame conversion
((895, 4), 895)
doen w getting raw video data 
All of the videos have been processed...


#### Checking the OCR first automatically, and then verifying it:
The step above should have given a .csv file with the following name : *videosync_[file path here]_.csv*. You will notice the 'timestamp_verified' columns is blank. This is because the effectiveness of the OCR can vary a lot.

#### IF you have a few timestamps to check (maybe ~100-1000)
Copy the 'timestamp' column into timestamp_verified and correct the entries wherever they look weird. It may be useful especially to check along transition points ie. where the second changes or the date changes. eg. from 2018-09-10 10:00:01 to 2018-09-10 10:00:02. The timestamp may remain as 2018-09-10 10:00:01 all throughout. 

This is also the point at which you can run *browse_through_video* and run through the videos and moving through the frame numbers of the weirdly read framenumbers.

#### IF you have a large number of timestamps to check (maybe >1000)
Beyond a certain number of rows, manual checking can get difficult - and you can use a very basic function that checks if things are broadly okay. This function is the 'detect_unusual_timestamps' from the 'check_OCR_timestamps' module.

The 'detect_unusual_timestamps' function works by checking if the timegap between adjacent timestamps makes sense in terms whether:
1) it is parsable. eg '2019-Ak-12 1234092144' is not parsable automatically to a timestamp, while 2019-09-10 12:34:09 is. 

2)makes sense in comparison to the timestamp just before it. eg. a frame with 2019-09-10 12:34:08 can be followed either by another frame with the same timestamp(12:34:09) or by a timestamp with one additional second in it (2019-09-10 12:34:09). Anything else indicates a bad OCR event or an actual jump in the timestamps observed. 

Wherever the order of read timestamps makes sense the ```user_suggestion``` is 'maybeokay', and whenever there are unparsable or odd timestamp jumps the ```user_suggestion``` is 'VERIFY'. Typically two neighbouring rows are marked with 'VERIFY'.



In [31]:
video_data = pd.read_csv('videosync_vlc-record-2019-08-19-16h32m09s-DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi_.csv')

In [32]:
video_data.head()

,Unnamed: 0,frame_number,led_intensity,timestamp,timestamp_verified
0,1,1,401853.0,2019-07-04 16:51:25,NaN
1,2,2,401757.0,2019-07-04 16:51:26,NaN
2,3,3,401736.0,2019-07-04 16:51:26,NaN
3,4,4,401688.0,2019-07-04 16:51:26,NaN
4,5,5,401673.0,2019-07-04 16:51:26,NaN


In [33]:
from check_OCR_timestamps import detect_unusual_timestamps 
user_suggestion = detect_unusual_timestamps(video_data['timestamp'], '%Y-%m-%d %H:%M:%S', seconds=1)

..........Checking for bad timestamps..........
..........Checking for odd-jumps between timestamps..........


In [34]:
# add user suggestion to the timestamp dataframe
timestamps_w_suggestion = pd.concat([video_data, user_suggestion], axis=1)

In [35]:
timestamps_w_suggestion

,Unnamed: 0,frame_number,led_intensity,timestamp,timestamp_verified,user_suggestion
0,1,1,401853.0,2019-07-04 16:51:25,NaN,maybeokay
1,2,2,401757.0,2019-07-04 16:51:26,NaN,maybeokay
2,3,3,401736.0,2019-07-04 16:51:26,NaN,maybeokay
3,4,4,401688.0,2019-07-04 16:51:26,NaN,maybeokay
4,5,5,401673.0,2019-07-04 16:51:26,NaN,maybeokay
5,6,6,401850.0,2019-07-04 16:51:26,NaN,maybeokay
6,7,7,401880.0,2019-07-04 16:51:26,NaN,maybeokay
7,8,8,401820.0,2019-07-04 16:51:26,NaN,maybeokay
8,9,9,401919.0,2019-07-04 16:51:26,NaN,maybeokay
9,10,10,401781.0,2019-07-04 16:51:26,NaN,maybeokay


In [36]:
np.unique(timestamps_w_suggestion['user_suggestion']) # quickly check 

array(['maybeokay'], dtype=object)

In [37]:
plt.figure()
plt.plot(video_data['led_intensity'])

In [40]:
# Load the corrected video_sync file again with a filled out 'timestamp_verified' column
checked_video_data = pd.read_csv('videosync_vlc-record-2019-08-19-16h32m09s-DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi_.csv')

### Resampling the led signal and checking for dropped frames :
The *videosync* file is raw data. Depending on your camera+DVR system there could be dropped frames or the framerate may vary over time. These issues need to be detected and corrected for at the level of each annotation. 

The output of this step is a series of *commonfps* files.

In [42]:
from process_video_annotations import video_sync_over_annotation_block

annotations = pd.read_csv('eg_annotations.csv')

kwargs = {'timestamp_pattern': '%Y-%m-%d %H:%M:%S'}
kwargs['min_fps']= 20 # Hz
kwargs['min_durn'] = 20.0 # seconds 
kwargs['common_fps'] = 25 # Hz
kwargs['output_folder'] = './'
success = annotations.apply(video_sync_over_annotation_block,1, video_sync_data=checked_video_data , 
                                 **kwargs)
print(success)


Saved annotation :555
0    True
dtype: bool


/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

In [43]:
annotations.head()

,video_path,annotation_id,start_timestamp,start_framenumber,end_timestamp,end_framenumber,timestamp_border,led_border
0,./vlc-record-2019-08-19-16h32m09s-DVRecorder_0...,555,2019-07-04 16:51:40,1,2019-07-04 16:51:50,15,"(587.2505532565692, 51.974969803362015, 112.16...","(416.26279113581603, 796.2260524351722, 496.60..."


In [3]:
def write_match_report(all_match_cc, all_files):
    df = pd.DataFrame(data={'file_name':all_files, 'match_cc':all_match_cc})
    now = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
    df.to_csv('AV-match-report_'+now+'.csv')
    

In [46]:
### Finding the matching audio snippet !! 
import glob 
import soundfile as sf
from audio_for_videoannotation import match_video_sync_to_audio

#all_commonfps = glob.glob('common_fps_video_sync*') # get all the relevant common_fps_sync files
all_commonfps = glob.glob('./common_fps*')
audio_folder = './audio/' # the current folder
audiosync_folder = './sync_audio/'
audioannotation_folder = './annotation_audio/'
fs = 500000 # change according to the recording sampling rate in Hz!! 
# generate the 

In [52]:
all_ccs = []
files_to_run = sorted(all_commonfps)
for somenumber, each_commonfps in enumerate(files_to_run):
    print(each_commonfps)
    video_sync = pd.read_csv(each_commonfps)
    best_audio, syncblock_audio, crosscoef = match_video_sync_to_audio(video_sync, audio_folder, audio_fileformat='*.WAV',
                                           audio_sync_spikey=False)
    all_ccs.append(crosscoef)
    fname  = os.path.split(each_commonfps)[-1]
    annotation_id = '-'.join(os.path.split(fname)[-1].split('_')[-2:])
                                                                 
    try:
        sf.write(audiosync_folder+'matching_sync_'+str(somenumber)+'.WAV', syncblock_audio,fs)
        sf.write(audioannotation_folder+'matching_annotaudio_'+str(somenumber)+'.WAV', best_audio,fs)
    except:
        print('Could not save ', each_commonfps)

#write_match_report(all_ccs, sorted(all_commonfps))


  0%|          | 0/9 [00:00<?, ?it/s]

./common_fps_video_sync555.csv
('video_fps obtained is :', 25)
Did not find user-provided sample rate - getting it from first file that matches format!
('sampling rate is : ', 250000)
.....finding best audio segment.....


100%|██████████| 9/9 [00:34<00:00,  3.83s/it]


('Best file pairs are: ', 'non_spikey_T0000681.WAV*non_spikey_T0000682.WAV')
Start and end indices around peak:                  start index: 5672717		          end index: 10922717                  total samples:30015488
